# Mask Generation Notebook
### by : Arun Chakravorty and Katsuya Colon

In [ ]:
#import functions
from cellpose_segmentation import *
import nuclear_cyto_match as ncm
import glob
import re

In [ ]:
# Load the images: 
input_directory = "/groups/CaiLab/personal/Lex/raw/111122_1knih3t3/segmentation/*.tif"
files=glob.glob(input_directory)
#organize files numerically
key = [int(re.search('MMStack_Pos(\\d+)', f).group(1)) for f in files]
files = list(np.array(files)[np.argsort(key)])

In [ ]:
#take a look at files
files

In [ ]:
#read in images
imgs = read_images(files, max_project=True)

In [ ]:
#Z, C, X, Y 
imgs[0].shape

## View Single File

In [ ]:
#which pos do you want to look at
pos=0
#do you want mask for each z?
have_multiple_z = False
#which channel has segmenation marker (0,1,2,3)?
channel = 1

img = isolate_image(imgs, pos = pos, channel = channel, have_multiple_z = have_multiple_z)

In [ ]:
#shape of isolated image
img.shape

In [ ]:
#contrast
zmax=5000

plot_isolated_image(img, have_multiple_z = have_multiple_z, zmax = zmax)

## Settings for Cellpose

In [ ]:
#run this to define settings
model = cellpose_settings(num_gpus=1)

### Test Cellpose on a Single Image
The ***Flow_threshold parameter*** is  the  maximum  allowed  error  of  the  flows  for  each  mask.   The  default  is 0.4. 

*   **Increase** this threshold if cellpose is not returning as many masks as you’d expect

*   **Decrease** this threshold if cellpose is returning too many ill-shaped masks. 

The ***Cell Probability Threshold*** determines proability that a detected object is a cell.   The  default  is 0.0. 

*   **Decrease** this threshold if cellpose is not returning as many masks as you’d expect or if masks are too small

*   **Increase** this threshold if cellpose is returning too many masks esp from dull/dim areas. 

In [ ]:
#channels to segment
channels = [0,0]

if have_multiple_z == True:
    masks, flows, styles, diams = model.eval(img, diameter=350, 
                                             channels=channels, flow_threshold=2,
                                             cellprob_threshold=-1, stitch_threshold=0.3, do_3D=False)
else:
    masks, flows, styles, diams = model.eval(img, diameter=350, 
                                             channels=channels, flow_threshold=2,
                                             cellprob_threshold=-1)

In [ ]:
num_channels = 2
cellpose_plots(img, masks, flows, have_multiple_z = have_multiple_z, num_channels = num_channels, channels=channels)

# Check to see if z's are properly stitched

In [ ]:
#np.unique(masks[0])

In [ ]:
#plt.imshow(masks[0]==95)

In [ ]:
#plt.imshow(masks[1]==95)

In [ ]:
#plt.imshow(masks[2]==95)

In [ ]:
#plt.imshow(masks[3]==95)

### Run Cellpose on Entire Directory 

In [ ]:
imgs_final = generate_final_images(imgs, have_multiple_z=have_multiple_z, channel=channel)

In [ ]:
if have_multiple_z == True:
    masks, flows, styles, diams = model.eval(imgs_final, diameter=350, 
                                             channels=channels, flow_threshold=2,
                                             cellprob_threshold=-1, stitch_threshold=0.3, do_3D=False)
else:
    masks, flows, styles, diams = model.eval(imgs_final, diameter=350, 
                                             channels=channels, flow_threshold=2,
                                             cellprob_threshold=-1, do_3D=False)

### Show Results 

In [ ]:
#pick example position
image_number = 1

if have_multiple_z == False:
    img1 = imgs_final[image_number]
    mask = masks[image_number]
    flow = flows[image_number]
    cellpose_plots(img1, mask, flow, have_multiple_z = have_multiple_z, 
                   num_channels = num_channels, channels=channels)
else:
    img1 = imgs_final[image_number]
    mask = masks[image_number]
    flow = flows[image_number]
    cellpose_plots(img1, mask, flow, have_multiple_z = have_multiple_z, 
                   num_channels = num_channels, channels=channels)

### Save Files 

In [ ]:
#name of output directory
save_dir = '/groups/CaiLab/personal/Lex/raw/111122_1knih3t3/seqFISH_datapipeline/output/labeled_images/cyto_masks/'

write_masks(masks, files, save_dir, repeat_mask_multi_z = 9)

# Match masks

In [ ]:
#read in masks
nuc_paths = glob.glob("/groups/CaiLab/personal/Lex/raw/090222_150genes_acrosschannel_3t3/notebook_pyfiles/labeled_images/nuc_masks_stitched/*.tif")
cyto_paths = glob.glob("/groups/CaiLab/personal/Lex/raw/090222_150genes_acrosschannel_3t3/notebook_pyfiles/labeled_images/cyto_masks_stitched/*.tif")

#organize files numerically
key = [int(re.search('MMStack_Pos(\\d+)', f).group(1)) for f in nuc_paths]
nuc_paths = list(np.array(nuc_paths)[np.argsort(key)])

key = [int(re.search('MMStack_Pos(\\d+)', f).group(1)) for f in cyto_paths]
cyto_paths = list(np.array(cyto_paths)[np.argsort(key)])

In [ ]:
#take a look
plt.imshow(tf.imread(cyto_paths[0]))

In [ ]:
#read in files
nuclear = []
cyto = []
for i in tqdm(range(len(nuc_paths))):
    nuclear.append(pil_imread(nuc_paths[i]))
    cyto.append(pil_imread(cyto_paths[i]))

In [ ]:
#match nuclear and cyto masks
cyto_new = ncm.nuclear_cyto_matching(cyto,nuclear, threshold=0.10)

In [ ]:
#take a look
plt.imshow(cyto_new[0])

In [ ]:
#write images
i=0
for lab_img in cyto_new:
    imagename = Path(nuc_paths[i]).name
    path = Path(f"../labeled_images/matched/{imagename}")
    path.parent.mkdir(parents=True, exist_ok=True)
    tf.imwrite(str(path),lab_img)
    i += 1